## 順位の実装  

##### モジュール

In [127]:
# device = "cuda:1"
device = "cuda:0"

In [11]:
import pickle

def save_bin(filename,data):
    with open(filename,'wb') as bf:
        pickle.dump(data,bf)

# def read_bin(filename):
#     with open(filename,'rb') as bf:
#         bin_data = pickle.load(bf)
#     return bin_data
    
def read_bin(filename):
    with open(filename, 'rb') as bf:
        bin_data = pickle.load(bf, encoding='shift_jis')  # ここで encoding を指定
    return bin_data

## 結果のソート

In [5]:
import pandas as pd

In [6]:
def Data_sorted(filename,test_index):
    data_sorted = []
    for idx in test_index:
        data = read_bin(filename+'/'+str(idx)+'.bin')
        data_sorted.append(dict(sorted(data.items(), key=lambda item: item[1],reverse=True)))
    return data_sorted

In [7]:
def Test_index(num):
    test = pd.read_parquet('BirdModel_remove_test/data_cross-valid/test_'+num+'.parquet')#10293
    return list(test.index)

単体モーダル

In [135]:
# for num in ["1","2","3","4","5"]:
#     test_index = Test_index(num)
    
#     # not p
#     data_sorted = Data_sorted('csv_cv'+num,test_index)
#     save_bin('data_sorted/cv'+num+'.bin',data_sorted)
    
#     #p
#     data_p_sorted = Data_sorted('csv_cv'+num+'_p',test_index)
#     save_bin('data_sorted/cv'+num+'_p.bin',data_p_sorted)

マルチモーダル

In [137]:
# for num in ["1","2","4","5"]:
for num in ["4","5"]:
    test_index = Test_index(num)
    for prm in ["L","G","LG","SG","SL"]:
        
        data_sorted = Data_sorted('csv_cv_'+prm+'_'+num,test_index)
        save_bin('data_sorted/'+prm+'_'+num+'.bin',data_sorted)

In [ ]:
import pickle

def is_pickle_file(filename):
    with open(filename, 'rb') as f:
        try:
            pickle.load(f)
            return True
        except pickle.UnpicklingError:
            return False

# 使用例
print(is_pickle_file('your_file.bin'))

In [12]:
# あとでまわす
for num in ["3"]:
    test_index = Test_index(num)
    for prm in ["LG","SG","SL"]:
        
        data_sorted = Data_sorted('csv_cv_'+prm+'_'+num,test_index)
        save_bin('data_sorted/'+prm+'_'+num+'.bin',data_sorted)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x8e in position 1: invalid start byte

In [157]:
prm

'LG'

In [ ]:
# あとでまわす
for num in ["3"]:
    test_index = Test_index(num)
    for prm in ["SG","SL"]:
        
        data_sorted = Data_sorted('csv_cv_'+prm+'_'+num,test_index)
        save_bin('data_sorted/'+prm+'_'+num+'.bin',data_sorted)

In [ ]:
# あとでまわす２
for num in ["1","2","3","4","5"]:
    test_index = Test_index(num)
    for prm in ["SLG"]:
        
        data_sorted = Data_sorted('csv_cv_'+prm+'_'+num,test_index)
        save_bin('data_sorted/'+prm+'_'+num+'.bin',data_sorted)

## 検索評価タスク

In [138]:
import pandas as pd
import numpy as np

In [139]:
def round_sf(x, sigfigs):
    if x == 0:
        return 0
    return round(x, sigfigs - int(np.floor(np.log10(abs(x)))) - 1)

In [140]:
def Test_ans(num):
    test = pd.read_parquet('BirdModel_remove_test/data_cross-valid/test_'+num+'.parquet')#10293
    WavDesc2Kana=read_bin('new_data/WavDesc2Kana.bin')
    name2jid=read_bin('new_data/name2jid.bin')
    tuples_bid_jid = read_bin('new_data/tuples_bid_jid.bin')
    
    test_ans = []
    for test_name in list(test["description"]):
        test_jid = name2jid[WavDesc2Kana[test_name]]
        for bid,jid in tuples_bid_jid:
            if jid == test_jid:
                test_ans.append(bid)
                break
    del test,WavDesc2Kana,name2jid,tuples_bid_jid
    return test_ans

In [141]:
def Get_sorted_id(data_sorted):
    data_sorted_id = []
    for d in data_sorted:
        data_sorted_id.append(list(d.keys()))
    return data_sorted_id

In [142]:
def Get_sorted_cos(data_sorted):
    data_sorted_cos = []
    for d in data_sorted:
        data_sorted_cos.append(list(d.values()))
    return data_sorted_cos

In [143]:
def Ranks(id_sorted,test_ans):
    ranks = []
    for i in range(len(test_ans)):
        if test_ans[i] in id_sorted[i]:
            index = id_sorted[i].index(test_ans[i])
            ranks.append(index+1)
        else:
            ranks.append(-1)
    return ranks

In [144]:
def AP(a_id_sorted,a_test_ans):#average_precision
    relevant_retrieved = 0
    precision_sum = 0.0
    
    for i, a_id in enumerate(a_id_sorted):
        if a_id == a_test_ans:
            relevant_retrieved += 1
            precision_sum += relevant_retrieved / (i + 1)
            
    return precision_sum / len(a_id_sorted)


def MAP(id_sorted,test_ans):#mean_average_precision
    ap_sum = 0.0
    for i in range(len(test_ans)):
        ap = AP(id_sorted[i],test_ans[i])
        ap_sum += ap
    map_score = ap_sum / len(test_ans)
    return round_sf(map_score,5)

In [145]:
def MRR(ranks):
    reciprocal_ranks = [1 / (rank + 1) for rank in ranks]
    mrr_score = sum(reciprocal_ranks) / len(ranks)
    return round_sf(mrr_score,5)

In [146]:
def Onehot(a_id_sorted,a_test_ans):
    return [1 if a_id == a_test_ans else 0 for a_id in a_id_sorted]

In [147]:
import numpy as np

def DCG(result_onehot,k):
    dcg_score = np.sum(np.asarray(result_onehot)[:k] / np.log2(np.arange(2, k + 2)))
    return round_sf(dcg_score,5)

def nDCG(result_onehot,k):
    ideal_onehot = sorted(result_onehot,reverse=True)
    ndcg_score = DCG(result_onehot, k) / DCG(ideal_onehot, k)
    return round_sf(ndcg_score,5)

In [148]:
import numpy as np
from sklearn.metrics import ndcg_score

def Mean_nDCG(id_sorted,cos_sorted,test_ans,k):
    ndcg_sum = 0.0
    for i in range(len(test_ans)):
        result_onehot = Onehot(id_sorted[i],test_ans[i])
        ndcg_sum += nDCG(result_onehot,k)

    mean_ndcg_score = ndcg_sum / len(test_ans)
    return round_sf(mean_ndcg_score,5)

In [149]:
# for num in ["1","2","3","4","5"]:
#     print("== "+num+" ===========")
#     test_ans = Test_ans(num)
    
#     data_sorted = read_bin('data_sorted/cv'+num+'.bin')
#     id_sorted = Get_sorted_id(data_sorted)
#     cos_sorted = Get_sorted_cos(data_sorted)
    
#     result_ranks = Ranks(id_sorted,test_ans)
    
#     print("MAP:",MAP(id_sorted,test_ans))
#     print("MMR:",MRR(result_ranks))
#     print("nDCG@10:",Mean_nDCG(id_sorted,cos_sorted,test_ans,10))
#     print("nDCG@50:",Mean_nDCG(id_sorted,cos_sorted,test_ans,50))
#     print("nDCG@100:",Mean_nDCG(id_sorted,cos_sorted,test_ans,100))

In [150]:
# #p
# for num in ["1","2","3","4","5"]:
#     print("== "+num+" ===========")
#     test_ans = Test_ans(num)
    
#     data_sorted = read_bin('data_sorted/cv'+num+'_p.bin')
#     id_sorted = Get_sorted_id(data_sorted)
#     cos_sorted = Get_sorted_cos(data_sorted)
    
#     result_ranks = Ranks(id_sorted,test_ans)
    
#     print("MAP:",MAP(id_sorted,test_ans))
#     print("MMR:",MRR(result_ranks))
#     print("nDCG@10:",Mean_nDCG(id_sorted,cos_sorted,test_ans,10))
#     print("nDCG@50:",Mean_nDCG(id_sorted,cos_sorted,test_ans,50))
#     print("nDCG@100:",Mean_nDCG(id_sorted,cos_sorted,test_ans,100))

In [159]:
# print("MAP MMR nDCG@10 nDCG@50 nDCG@100")
# for prm in ["L","G","LG","SG","SL"]:
#     print(prm)
#     for num in ["1","2","4","5"]:
#         test_ans = Test_ans(num)
        
#         data_sorted = read_bin('data_sorted/'+prm+'_'+num+'.bin')
#         id_sorted = Get_sorted_id(data_sorted)
#         cos_sorted = Get_sorted_cos(data_sorted)
        
#         result_ranks = Ranks(id_sorted,test_ans)
#         print(num,MAP(id_sorted,test_ans),MRR(result_ranks),Mean_nDCG(id_sorted,cos_sorted,test_ans,10),Mean_nDCG(id_sorted,cos_sorted,test_ans,50),Mean_nDCG(id_sorted,cos_sorted,test_ans,100))

In [160]:
print("MAP MMR nDCG@10 nDCG@50 nDCG@100")
for prm in ["L","G"]:
    print(prm)
    for num in ["3"]:
        test_ans = Test_ans(num)
        
        data_sorted = read_bin('data_sorted/'+prm+'_'+num+'.bin')
        id_sorted = Get_sorted_id(data_sorted)
        cos_sorted = Get_sorted_cos(data_sorted)
        
        result_ranks = Ranks(id_sorted,test_ans)
        print(num,MAP(id_sorted,test_ans),MRR(result_ranks),Mean_nDCG(id_sorted,cos_sorted,test_ans,10),Mean_nDCG(id_sorted,cos_sorted,test_ans,50),Mean_nDCG(id_sorted,cos_sorted,test_ans,100))

MAP MMR nDCG@10 nDCG@50 nDCG@100
L
3 1.82e-05 0.47042 0.93761 0.9406 0.9406
G
3 1.2901e-05 0.35924 0.71147 0.72972 0.73075


In [ ]:
# あとでまわす
print("MAP MMR nDCG@10 nDCG@50 nDCG@100")
for prm in ["LG","SG","SL"]:
    print(prm)
    for num in ["3"]:
        test_ans = Test_ans(num)
        
        data_sorted = read_bin('data_sorted/'+prm+'_'+num+'.bin')
        id_sorted = Get_sorted_id(data_sorted)
        cos_sorted = Get_sorted_cos(data_sorted)
        
        result_ranks = Ranks(id_sorted,test_ans)
        print(num,MAP(id_sorted,test_ans),MRR(result_ranks),Mean_nDCG(id_sorted,cos_sorted,test_ans,10),Mean_nDCG(id_sorted,cos_sorted,test_ans,50),Mean_nDCG(id_sorted,cos_sorted,test_ans,100))

In [ ]:
# あとでまわす
print("MAP MMR nDCG@10 nDCG@50 nDCG@100")
for prm in ["SLG"]:
    print(prm)
    for num in ["1","2","3","4","5"]:
        test_ans = Test_ans(num)
        
        data_sorted = read_bin('data_sorted/'+prm+'_'+num+'.bin')
        id_sorted = Get_sorted_id(data_sorted)
        cos_sorted = Get_sorted_cos(data_sorted)
        
        result_ranks = Ranks(id_sorted,test_ans)
        print(num,MAP(id_sorted,test_ans),MRR(result_ranks),Mean_nDCG(id_sorted,cos_sorted,test_ans,10),Mean_nDCG(id_sorted,cos_sorted,test_ans,50),Mean_nDCG(id_sorted,cos_sorted,test_ans,100))

In [ ]:
# 音声の何パーセントがなくなっているのか
# ランダムシャッフルした検索順位での比較実験
# epoch = 80
# 0.12以上
# setp数はシャッフルのあと
# それよりは
# 20%でテスト（5ホールド前提）